# Simple Keras CNN for mnist to use as feature extractor and benchmark

In [1]:
from keras.datasets import mnist
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.layers import Input
from keras.optimizers import Adadelta
from keras.utils import np_utils
from keras import backend as K
from sklearn.metrics import mean_squared_error

import tensorflow as tf

Using TensorFlow backend.


In [2]:
K.set_image_dim_ordering('th')
tf.python.control_flow_ops = tf

In [3]:
img_rows, img_cols = 28, 28
nb_filters = 32
pool_size = (2, 2)
kernel_size = (3, 3)

In [4]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [5]:
X_train = X_train.reshape(X_train.shape[0], 1, img_rows, img_cols)
X_test = X_test.reshape(X_test.shape[0], 1, img_rows, img_cols)
input_shape = (1, img_rows, img_cols)

In [6]:
inputs = Input(shape=input_shape)
conv1 = Activation('relu')(Convolution2D(nb_filters, kernel_size[0], kernel_size[1],
                        border_mode='valid',
                        input_shape=input_shape)(inputs))
conv2 = Activation('relu')(Convolution2D(nb_filters, kernel_size[0], kernel_size[1])(conv1))
pool1 = Dropout(0.25)(MaxPooling2D(pool_size=pool_size)(conv2))
flat = Flatten()(pool1)
dense1 = Dropout(0.5)(Activation('relu')(Dense(64)(flat)))
prediction = Dense(1)(dense1)

model = Model(input=inputs, output=prediction)
feature_extractor = Model(input=inputs, output=dense1)

In [8]:
model.compile(loss='mse',
              optimizer=Adadelta(lr=1.),
              metrics=[])

In [9]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 1, 28, 28)     0                                            
____________________________________________________________________________________________________
convolution2d_1 (Convolution2D)  (None, 32, 26, 26)    320         input_1[0][0]                    
____________________________________________________________________________________________________
activation_1 (Activation)        (None, 32, 26, 26)    0           convolution2d_1[0][0]            
____________________________________________________________________________________________________
convolution2d_2 (Convolution2D)  (None, 32, 24, 24)    9248        activation_1[0][0]               
___________________________________________________________________________________________

In [10]:
model.fit(X_train, y_train, batch_size=64, nb_epoch=2,
          verbose=1, validation_data=(X_test, y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/2
60000/60000 [==============================] - 274s - loss: 11.8698 - val_loss: 1.6959
Epoch 2/2
60000/60000 [==============================] - 305s - loss: 3.2863 - val_loss: 1.2128


In [11]:
preds = model.predict(X_test)

In [12]:
preds_tr = model.predict(X_train)

In [13]:
mean_squared_error(y_train, preds_tr) 

1.2011201785952934

In [14]:
mean_squared_error(y_test, preds)

1.2128070156745305

In [36]:
model.get_weights()[0].shape

(32, 1, 3, 3)

In [15]:
features_test = feature_extractor.predict(X_test)

In [17]:
features_train = feature_extractor.predict(X_train)

In [16]:
features_test.shape

(10000, 64)

In [18]:
features_train.shape

(60000, 64)

In [20]:
import numpy as np

In [21]:
np.save('features_tr', features_train)
np.save('features_te', features_test)